# Finding the best version of the kernel function

We have 6 different approaches ranging on verbose/iteration vs vector operations, each with a sub-variant modified to work with Numba to (hopefully) speed up the computation.

## Versions
1. `compute_k` - verbose, iteration version
2. `compute_k_half_np` - half vectorized version
3. `compute_k_np` - fully vectorized version
4. `compute_k_` - variation of (1)
5. `compute_k_half_np_` - variation of (2)
6. `compute_k_np_` - variation of (3)

For each version, need to verify that:
1. Numba version matches the regular version
2. Both produce a correct result (compare with verified version where possible) 

TODO: which versions should produce the same result? Should the first 3 and second three all be the same?

In [113]:
# print("lambdas: ", method.lambdas)
# print("CV MSEs: ", method.cv_mses)

# # plot cv mse as function of lambda using altair
# import altair as alt
# import pandas as pd

# # Create a DataFrame from the cross-validated MSEs
# df = pd.DataFrame({'Lambda': method.lambdas, 'CV MSE': method.cv_mses})

# # Create a line plot of CV MSE as a function of lambda
# line_chart = alt.Chart(df).mark_line(point=True).encode(
#     x='Lambda:Q',
#     y='CV MSE:Q'
# ).properties(
#     title='Cross-Validated MSE as a Function of Lambda',
#     width=400,
#     height=200
# )

# # Display the line chart
# line_chart.display()

## Setup: getting a subset of data to test kernel function variations

In [114]:

from highly_adaptive_lasso import HAL
from highly_adaptive_ridge import HAR
from kernel_har import KernelHAR
import numpy as np
import pandas as pd 
from run_trials import RunTrials
import warnings
from train_time_plotter import TrainTimePlotter
from sklearn.model_selection import train_test_split

dataset = "kin8nm"
data = pd.read_csv(f"/Users/alexhagemeister/Documents/RWM/HAR/Highly_Adaptive_Ridge/csv/{dataset}.csv")
# Last column is the target
X = data.iloc[:, :-1].values
Y = data.iloc[:, -1].values

# get the first 200 samples

n = 200
X = X[:n,:]
Y = Y[:n]

# split into training (X) and validation (X') sets
X_train, X_prime, Y_train, Y_prime = train_test_split(X, Y, test_size=0.2, random_state=42)

# har = HAR()
# har.knots = X_train
# basis_matrix = har._bases(X_train)

# K_har = basis_matrix @ basis_matrix.T

In [115]:
# create dictionary to store training time results for each kernel fuinction variation
train_times = {}

# 1) compute_k (verbose)

I believe this one was verified to be correct, but need to check

## 1.1 compute_k

In [116]:
def compute_K(X_train, X_prime):
    K = np.empty((
        X_train.shape[0], # j: training observation
        X_prime.shape[0], # k: test observation
    ), dtype=int)
    for tr in range(X_train.shape[0]):
        for te in range(X_prime.shape[0]):
                K[tr, te] = sum(
                    2**(np.sum(
                        X_train[knot,:] <= np.minimum(X_train[tr, :], X_prime[te, :])
                    )) for knot in range(X_train.shape[0])
                )
    return K

from time import time
start = time()
K = compute_K(X_train, X_train)
train_times["compute_K"] = time() - start

# print leaderboard (the train_times dictionary sorted by value)
print(f"Train Times at n={n}")
print("-----------")
for key in train_times:
    print(f"{key}: {train_times[key]}")



Train Times at n=200
-----------
compute_K: 10.181051969528198


## 1.2 compute_k with Numba

In [117]:
from numba import jit, njit

@njit
def compute_K_numba(X_train, X_prime):
    n_train = X_train.shape[0]
    n_prime = X_prime.shape[0]
    n_features = X_train.shape[1]
    
    K = np.empty((n_train, n_prime), dtype=np.int64)
    equal = True
    for tr in range(n_train):
        if equal:
            max_index = tr + 1
        else:
            max_index = n_prime
        for te in range(max_index):
            sum_val = 0
            for knot in range(n_train):
                count = 0
                for feature in range(n_features):
                    if X_train[knot, feature] <= min(X_train[tr, feature], X_prime[te, feature]):
                        count += 1
                sum_val += 2 ** count
            K[tr, te] = sum_val
            if equal:
                K[te, tr] = sum_val
            
    return K

start = time()
K_numba = compute_K_numba(X_train, X_train)
train_times["compute_K_numba"] = time() - start

print(f"Train Times at n={n}")
print("-----------")
for key in train_times:
    print(f"{key}: {train_times[key]}")


Train Times at n=200
-----------
compute_K: 10.181051969528198
compute_K_numba: 0.11459517478942871


## 1.3 verify results

In [118]:
# verify that the two implementations are the same
print("K == K_numba: ", np.allclose(K, K_numba))
K == K_numba

K == K_numba:  True


array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]])

---

# 2. compute_k_half_np 
with and without numba

## 2.1 compute_k_half_np reg

In [119]:
def compute_K_half_np(X_train, X_prime):
    min_matrix = np.minimum(X_train[:, np.newaxis, :], X_prime[np.newaxis, :, :])
    
    return sum(
        2 ** np.sum(X_train[knot,:] <= min_matrix[np.newaxis,:,:,:], axis=-1)
        for knot in range(X_train.shape[0])
    )

start = time()
K_half_np = compute_K_half_np(X_train, X_prime)
train_times["compute_K_half_np"] = time() - start

print(f"Train Times at n={n}")
print("-----------")
for key in train_times:
    print(f"{key}: {train_times[key]}")


Train Times at n=200
-----------
compute_K: 10.181051969528198
compute_K_numba: 0.11459517478942871
compute_K_half_np: 0.02012181282043457


## 2.2 compute_k_half_np with Numba

In [120]:
# TODO: fix the numba implementation
from numba import njit

@njit
def compute_K_half_np_numba(X_train, X_prime):
    min_matrix = np.minimum(X_train[:, np.newaxis, :], X_prime[np.newaxis, :, :])

    knot_sums = [2 ** np.sum(X_train[knot,:] <= min_matrix[np.newaxis,:,:,:], axis=-1) for knot in range(X_train.shape[0])]

    total_sums = sum(knot_sums) 
    
    return total_sums

start = time()
K_half_np_numba = compute_K_half_np_numba(X_train, X_prime)

train_times["compute_K_half_np_numba"] = time() - start

print(f"Train Times at n={n}")
print("-----------")
for key in train_times:
    print(f"{key}: {train_times[key]}")



TypingError: Failed in nopython mode pipeline (step: nopython frontend)
No implementation of function Function(<built-in function sum>) found for signature:
 
 >>> sum(list(array(int64, 3d, C))<iv=None>)
 
There are 2 candidate implementations:
      - Of which 2 did not match due to:
      Overload in function 'ol_sum': File: numba/cpython/builtins.py: Line 692.
        With argument(s): '(list(array(int64, 3d, C))<iv=None>)':
       Rejected as the implementation raised a specific error:
         TypingError: Failed in nopython mode pipeline (step: nopython frontend)
       Cannot unify Literal[int](0) and array(int64, 3d, C) for 'acc.2', defined at /Users/alexhagemeister/.local/share/virtualenvs/Highly_Adaptive_Ridge-0AT-AwqX/lib/python3.11/site-packages/numba/cpython/builtins.py (721)
       
       File "../../../../.local/share/virtualenvs/Highly_Adaptive_Ridge-0AT-AwqX/lib/python3.11/site-packages/numba/cpython/builtins.py", line 721:
           def impl(iterable, start=0):
    

## 2.3 verify results

In [ ]:
# verify Numba version matches
print("K_half_np == K_half_np_numba: ", np.allclose(K_half_np, K_half_np_numba))
K_half_np == K_half_np_numba

In [ ]:
# verify same as verified verbose version (K)
print("K_half_np == K: ", np.allclose(K_half_np, K))
K_half_np == K

---

# 3. `compute_k_np` 
with and without numba

## 3.1 compute_k_np reg

In [ ]:
# No numba
def compute_K_np(X_train, X_prime):
    min_matrix = np.minimum(X_train[:, np.newaxis, :], X_prime[np.newaxis, :, :])
    
    comparison_sum = (X_train[:, np.newaxis, np.newaxis, :] <= min_matrix).sum(axis=-1)
    return np.sum(2**comparison_sum, axis=0)

start = time()
K_np = compute_K_np(X_train, X_prime)
train_times["compute_K_np"] = time() - start

print(f"Train Times at n={n}")
print("-----------")
for key in train_times:
    print(f"{key}: {train_times[key]}")

Train Times at n=200
-----------
compute_K: 10.064973831176758
compute_K_numba: 0.2024211883544922
compute_K_half_np: 0.03492569923400879
compute_K_np: 0.034459829330444336
compute_K_np_numba: 1.0132229328155518
compute_K_: 2.167156934738159
compute_K_numba_: 0.3313629627227783
compute_K_half_np_numba: 0.05913496017456055


## 3.2 compute_k_np with Numba

In [ ]:
from numba import njit

@njit
def compute_K_np_numba(X_train, X_prime):
    min_matrix = np.minimum(X_train[:, np.newaxis, :], X_prime[np.newaxis, :, :])
    
    comparison_sum = (X_train[:, np.newaxis, np.newaxis, :] <= min_matrix).sum(axis=-1)
    return np.sum(2**comparison_sum, axis=0)

start = time()
K_np_numba = compute_K_np_numba(X_train, X_prime)
train_times["compute_K_np_numba"] = time() - start

print(f"Train Times at n={n}")
print("-----------")
for key in train_times:
    print(f"{key}: {train_times[key]}")

Train Times at n=200
-----------
compute_K: 10.064973831176758
compute_K_numba: 0.2024211883544922
compute_K_half_np: 0.03492569923400879
compute_K_np: 0.034459829330444336
compute_K_np_numba: 0.7602951526641846
compute_K_: 2.167156934738159
compute_K_numba_: 0.3313629627227783
compute_K_half_np_numba: 0.05913496017456055


## 3.3 verify results

In [ ]:
# compare normal vs numba
print("K_np == K_np_numba: ", np.allclose(K_np, K_np_numba))
K_np == K_np_numba

K_np == K_np_numba:  True


array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]])

In [ ]:
# verify same as verified verbose version (K)
print("K_np == K: ", np.allclose(K_np, K))
K_np == K

ValueError: operands could not be broadcast together with shapes (160,40) (160,160) 

---

# 4. `compute_K_`

## 4.1 compute_K_ regular

In [ ]:
# computing mins_train and mins_prime
# No numba
def compute_K_(X_train, X_prime):
    mins_train, mins_prime = (X_train[:, np.newaxis, :] <= X for X in (X_train, X_prime))

    K = np.empty((
        X_train.shape[0], # j: training observation
        X_prime.shape[0], # k: test observation
    ), dtype=int)
    for tr in range(X_train.shape[0]):
        for te in range(X_prime.shape[0]):
                K[tr, te] = sum(
                    2**(np.sum(mins_train[knot, tr, :] & mins_prime[knot, te, :])) 
                    for knot in range(X_train.shape[0])
                )
    return K

start = time()
K_ = compute_K_(X_train, X_prime)
train_times["compute_K_"] = time() - start

print(f"Train Times at n={n}")
print("-----------")
for key in train_times:
    print(f"{key}: {train_times[key]}")

Train Times at n=200
-----------
compute_K: 10.064973831176758
compute_K_numba: 0.2024211883544922
compute_K_half_np: 0.03492569923400879
compute_K_np: 0.034459829330444336
compute_K_np_numba: 0.7602951526641846
compute_K_: 2.1626009941101074
compute_K_numba_: 0.3142690658569336
compute_K_half_np_numba: 0.05913496017456055
compute_K_half_np_: 0.03696894645690918
compute_K_half_np_numba_: 0.47379207611083984
compute_K_np_: 0.03054213523864746
compute_K_np_numba_: 0.06004619598388672


## 4.2 compute_K_ with Numba

In [ ]:
from numba import njit

@njit
def compute_K_numba_(X_train, X_prime):
    mins_train = X_train[:, np.newaxis, :] <= X_train
    mins_prime = X_train[:, np.newaxis, :] <= X_prime

    K = np.empty((
        X_train.shape[0],  # j: training observation
        X_prime.shape[0],  # k: test observation
    ), dtype=np.int64)  # use np.int64 for compatibility with numba
    
    for tr in range(X_train.shape[0]):
        for te in range(X_prime.shape[0]):
            total_sum = 0
            for knot in range(X_train.shape[0]):
                condition_sum = np.sum(mins_train[knot, tr, :] & mins_prime[knot, te, :])
                total_sum += 2 ** condition_sum
            K[tr, te] = total_sum
    
    return K

start = time()
K_numba_ = compute_K_numba_(X_train, X_prime)
train_times["compute_K_numba_"] = time() - start

print(f"Train Times at n={n}")
print("-----------")
for key in train_times:
    print(f"{key}: {train_times[key]}")

Train Times at n=200
-----------
compute_K: 10.064973831176758
compute_K_numba: 0.2024211883544922
compute_K_half_np: 0.03492569923400879
compute_K_np: 0.034459829330444336
compute_K_np_numba: 0.7602951526641846
compute_K_: 2.1626009941101074
compute_K_numba_: 0.4317128658294678
compute_K_half_np_numba: 0.05913496017456055
compute_K_half_np_: 0.03696894645690918
compute_K_half_np_numba_: 0.47379207611083984
compute_K_np_: 0.03054213523864746
compute_K_np_numba_: 0.06004619598388672


## 4.3 verify results

In [121]:
# verify regular == numba
print("K_ == K_numba_: ", np.allclose(K_, K_numba_))
K_ == K_numba_

K_ == K_numba_:  True


array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]])

In [ ]:
# verify same as verified verbose version
print("K_ == K: ", np.allclose(K_, K))
K_ == K

ValueError: operands could not be broadcast together with shapes (160,40) (160,160) 

---

# 5. `compute_k_half_np_` 

## 5.1 compute_k_half_np_ regular

In [ ]:
def compute_K_half_np_(X_train, X_prime):
    mins_train, mins_prime = (X_train[:, np.newaxis, :] <= X for X in (X_train, X_prime))
    return sum(
        2 ** np.sum(mins_train[knot, :, np.newaxis, :] & mins_prime[knot, np.newaxis, :, :], axis=-1)
        for knot in range(X_train.shape[0])
    )

start = time()
K_half_np_ = compute_K_half_np_(X_train, X_prime)
train_times["compute_K_half_np_"] = time() - start

print(f"Train Times at n={n}")
print("-----------")
for key in train_times:
    print(f"{key}: {train_times[key]}")

Train Times at n=200
-----------
compute_K: 10.064973831176758
compute_K_numba: 0.2024211883544922
compute_K_half_np: 0.03492569923400879
compute_K_np: 0.034459829330444336
compute_K_np_numba: 0.7602951526641846
compute_K_: 2.1626009941101074
compute_K_numba_: 0.4317128658294678
compute_K_half_np_numba: 0.05913496017456055
compute_K_half_np_: 0.02832508087158203
compute_K_half_np_numba_: 0.47379207611083984
compute_K_np_: 0.03054213523864746
compute_K_np_numba_: 0.06004619598388672


## 5.2 compute_k_half_np_ with Numba

In [ ]:
# TODO: fix the numba implementation
from numba import njit

@njit
def compute_K_half_np_numba_(X_train, X_prime):
    mins_train = X_train[:, np.newaxis, :] <= X_train
    mins_prime = X_train[:, np.newaxis, :] <= X_prime
    
    total_sum = 0
    for knot in range(X_train.shape[0]):
        condition_matrix = mins_train[knot, :, np.newaxis, :] & mins_prime[knot, np.newaxis, :, :]
        sum_val = np.sum(condition_matrix, axis=-1)
        total_sum += np.sum(2 ** sum_val)
    
    return total_sum

start = time()
K_half_np_numba_ = compute_K_half_np_numba_(X_train, X_prime)
train_times["compute_K_half_np_numba_"] = time() - start

print(f"Train Times at n={n}")
print("-----------")
for key in train_times:
    print(f"{key}: {train_times[key]}")

Train Times at n=200
-----------
compute_K: 10.064973831176758
compute_K_numba: 0.2024211883544922
compute_K_half_np: 0.03492569923400879
compute_K_np: 0.034459829330444336
compute_K_np_numba: 0.7602951526641846
compute_K_: 2.1626009941101074
compute_K_numba_: 0.4317128658294678
compute_K_half_np_numba: 0.05913496017456055
compute_K_half_np_: 0.02832508087158203
compute_K_half_np_numba_: 0.4782600402832031
compute_K_np_: 0.03054213523864746
compute_K_np_numba_: 0.06004619598388672


## 5.3 verify results

In [ ]:
# verify normal == numba 
print("K_half_np_ == K_half_np_numba_: ", np.allclose(K_half_np_, K_half_np_numba_))
K_half_np_ == K_half_np_numba_

K_half_np_ == K_half_np_numba_:  False


array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

ERROR: numba version is not the same as numpy version

---

# 6. `compute k_np_`

## 6.1 compute_k_np_ regular

In [ ]:
def compute_K_np_(X_train, X_prime):
    mins_train, mins_prime = (X_train[:, np.newaxis, :] <= X for X in (X_train, X_prime))
    
    comparison_sum = (mins_train[:, :, np.newaxis, :] & mins_prime[:, np.newaxis, :, :]).sum(axis=-1)
    return np.sum(2**comparison_sum, axis=0)

start = time()
K_np_ = compute_K_np_(X_train, X_prime)
train_times["compute_K_np_"] = time() - start

print(f"Train Times at n={n}")
print("-----------")
for key in train_times:
    print(f"{key}: {train_times[key]}")

Train Times at n=200
-----------
compute_K: 10.064973831176758
compute_K_numba: 0.2024211883544922
compute_K_half_np: 0.03492569923400879
compute_K_np: 0.034459829330444336
compute_K_np_numba: 0.7602951526641846
compute_K_: 2.1626009941101074
compute_K_numba_: 0.4317128658294678
compute_K_half_np_numba: 0.05913496017456055
compute_K_half_np_: 0.02832508087158203
compute_K_half_np_numba_: 0.4782600402832031
compute_K_np_: 0.030025959014892578
compute_K_np_numba_: 0.06004619598388672


## 6.2 compute_k_np_ with Numba

In [ ]:
from numba import njit

@njit
def compute_K_np_numba_(X_train, X_prime):
    mins_train = X_train[:, np.newaxis, :] <= X_train
    mins_prime = X_train[:, np.newaxis, :] <= X_prime

    comparison_sum = np.empty((X_train.shape[0], X_train.shape[0], X_prime.shape[0]), dtype=np.int64)
    
    for i in range(X_train.shape[0]):
        for j in range(X_train.shape[0]):
            for k in range(X_prime.shape[0]):
                comparison_sum[i, j, k] = np.sum(mins_train[i, j, :] & mins_prime[i, k, :])
                
    return np.sum(2 ** comparison_sum, axis=0)

start = time()
K_np_numba_ = compute_K_np_(X_train, X_prime)
train_times["compute_K_np_numba_"] = time() - start

print(f"Train Times at n={n}")
print("-----------")
for key in train_times:
    print(f"{key}: {train_times[key]}")

Train Times at n=200
-----------
compute_K: 10.064973831176758
compute_K_numba: 0.2024211883544922
compute_K_half_np: 0.03492569923400879
compute_K_np: 0.034459829330444336
compute_K_np_numba: 0.7602951526641846
compute_K_: 2.1626009941101074
compute_K_numba_: 0.4317128658294678
compute_K_half_np_numba: 0.05913496017456055
compute_K_half_np_: 0.02832508087158203
compute_K_half_np_numba_: 0.4782600402832031
compute_K_np_: 0.030025959014892578
compute_K_np_numba_: 0.031881093978881836


## 6.3 verify results

In [ ]:
# verify normal == numba
print("K_np_ == K_np_numba_: ", np.allclose(K_np_, K_np_numba_))
K_np_ == K_np_numba_

K_np_ == K_np_numba_:  True


array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]])

In [ ]:
# verify same as verified verbose version (K)
print("K_np_ == K: ", np.allclose(K_np_, K))
K_np_ == K


ValueError: operands could not be broadcast together with shapes (160,40) (160,160) 

# Boneyard + workbench below (random code snippets)

In [ ]:
compute_K(X_train, X_prime) == compute_K_np(X_train, X_prime)

In [ ]:
knot = 11
tr = 50
te = 10

min_matrix = np.minimum(X_train[:, np.newaxis, :], X_prime[np.newaxis, :, :])

np.sum(X_train[knot,:] <= min_matrix[tr, te, :])

4

In [ ]:
comparison_sum = (X_train[:, np.newaxis, np.newaxis, :] <= min_matrix).sum(axis=-1)
comparison_sum[knot, tr, te]

4

In [ ]:
X_train[knot, :] < min_matrix[tr, te, :]

array([False, False, False, False, False,  True, False,  True])

In [ ]:
mins_train = X_train[:, np.newaxis, :] <= X_train
mins_prime = X_train[:, np.newaxis, :] <= X_prime

In [ ]:
sums = np.empty((X_train.shape[0], X_train.shape[0], X_prime.shape[0]), dtype=int)
for i in range(X_train.shape[0]):
    for j in range(X_train.shape[0]):
        for k in range(X_prime.shape[0]):
            sums[i,j,k] = np.sum(mins_train[i,j,:] & mins_prime[i,k,:])

KeyboardInterrupt: 

In [ ]:
K == K_fast

array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]])

In [ ]:
comparison_sorta_fast_sum = np.empty((X_train.shape[0], X_train.shape[0], X_prime.shape[0]), dtype=bool)
for i in range(X_train.shape[0]):
    comparison_sorta_fast_sum[i, :, :] = np.sum(X_train[i, np.newaxis, :] <= min_matrix, axis=-1)

In [ ]:
comparison_sorta_fast_sum == comparison_fast.sum(axis=-1)

array([[[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False,  True, ...,  True, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, Fal

In [ ]:
K = np.sum(2**comparison_sorta_fast_sum, axis=0)

In [ ]:
K_ = np.sum(2**np.sum(comparison_fast, axis=-1), axis=0)

In [ ]:
np.sum(K != K_)

6400

In [ ]:
np.all(comparison_fast == comparison_sorta_fast)

True

In [ ]:
comparison_sum = comparison.sum(axis=-1)

In [ ]:
exp_comparison = 2**comparison_sum

In [ ]:
K = np.sum(exp_comparison, axis=0)

In [ ]:
one_way_bases = np.stack([
    np.less.outer(X[:,j], X_prime[:,j])
    for j in range(X.shape[1])
])
one_way_bases.shape

(8, 200, 40)

In [ ]:
X.shape, X_prime.shape

((200, 8), (40, 8))

In [ ]:
basis_matrix.shape

(160, 40800)

In [ ]:
X.shape[0]* (2**(X.shape[1])-1)

51000

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
# from sklearn.preprocessing import StandardScaler
# from tabulate import tabulate
import time

# Sample data (replace with actual data)
data = {
    'data': ['yacht', 'energy', 'boston', 'concrete', 'wine', 'power', 'kin8nm', 'naval', 'protein', 'blog', 'slice', 'yearmsd'],
    'p': [6, 8, 13, 8, 11, 4, 8, 17, 9, 280, 384, 90],
    'n': [308, 768, 506, 1030, 1599, 9568, 8192, 11934, 45730, 52397, 53500, 515345],
    'LTB': ['0.90 (10.69)', '0.40 (30.82)', '3.35 (5.92)', '4.70 (43.29)', '0.64 (6.01)', '3.41 (56.92)', '0.12 (96.50)', '0.00 (107.98)', '1.94 (611.38)', '23.49 (185.49)', '1.23 (3350.61)', '8.54 (4616.82)'],
    'GBT': ['0.90 (4.68)', '0.40 (21.46)', '3.43 (4.47)', '4.87 (37.15)', '0.63 (3.58)', '3.46 (28.88)', '0.10 (60.40)', '0.00 (56.19)', '1.94 (96.80)', '23.46 (9.90)', '1.24 (3067.95)', '8.54 (1543.05)'],
    'HAL': ['0.72 (0.92)', '0.43 (45.80)', '3.66 (916.61)', '4.02 (134.01)', '–', '–', '–', '–', '–', '–', '–', '–'],
    'LASSO': ['8.92 (0.01)', '4.14 (0.01)', '5.02 (0.01)', '10.40 (0.01)', '0.67 (0.03)', '4.59 (0.04)', '0.21 (0.03)', '0.01 (10.51)', '2.50 (0.33)', '28.25 (13.51)', '8.33 (121.71)', '9.49 (40.09)'],
    'HAR': ['–'] * 12,  # Initialize with '-'
    'KernelHAR': ['–'] * 12  # Initialize with '-'
}

# Create DataFrame
table_df = pd.DataFrame(data)

# pickle the dataframe as 'models_compare_table.pickle'
table_file_name = "models_compare_table.pickle"
df.to_pickle(table_file_name)

# Display the DataFrame
table_df

,data,p,n,LTB,GBT,HAL,LASSO,HAR,KernelHAR
0,yacht,6,308,0.90 (10.69),0.90 (4.68),0.72 (0.92),8.92 (0.01),–,–
1,energy,8,768,0.40 (30.82),0.40 (21.46),0.43 (45.80),4.14 (0.01),–,–
2,boston,13,506,3.35 (5.92),3.43 (4.47),3.66 (916.61),5.02 (0.01),–,–
3,concrete,8,1030,4.70 (43.29),4.87 (37.15),4.02 (134.01),10.40 (0.01),–,–
4,wine,11,1599,0.64 (6.01),0.63 (3.58),–,0.67 (0.03),–,–
5,power,4,9568,3.41 (56.92),3.46 (28.88),–,4.59 (0.04),–,–
6,kin8nm,8,8192,0.12 (96.50),0.10 (60.40),–,0.21 (0.03),–,–
7,naval,17,11934,0.00 (107.98),0.00 (56.19),–,0.01 (10.51),–,–
8,protein,9,45730,1.94 (611.38),1.94 (96.80),–,2.50 (0.33),–,–
9,blog,280,52397,23.49 (185.49),23.46 (9.90),–,28.25 (13.51),–,–


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from IPython.display import display

def update_results(df, dataset_name, method_name, rmse, training_time):
    index = df[df['data'] == dataset_name].index[0]
    df.at[index, method_name] = f"{rmse:.2f} ({training_time:.2f})"

# Sample function to simulate training and testing
def train_and_evaluate(method, X_train, Y_train, X_test, Y_test):
    start_time = time.time()
    method.fit(X_train, Y_train)
    training_time = time.time() - start_time
    predictions = method.predict(X_test)
    mse = mean_squared_error(Y_test, predictions)
    rmse = np.sqrt(mse)
    return rmse, training_time

# funtion to read data from csv file and return X and Y
def read_data(file_path):
    data = pd.read_csv(file_path)
    # Check for missing values
    if data.isnull().values.any():
        print("Data contains missing values. Please handle them before training.")
        data = data.dropna()

    # Ensure all data are numeric
    if not all(np.issubdtype(dtype, np.number) for dtype in data.dtypes):
        print("Data contains non-numeric values. Please convert them to numeric before training.")

    # Last column is the target
    X = data.iloc[:, :-1].values
    Y = data.iloc[:, -1].values
    return X, Y

def run_real_data_trials(dataset_name, method, num_trials=1):
    """
    Run trials on a real dataset using the specified method.
    PARAM: 
        dataset_name: str - name of the dataset 
        method: object - the method to use
    RETURN:
        average_rmse: float - the average RMSE over all trials
        average_training_time: float - the average training time over all trials
    """

    # Load the data and split into X and Y
    X, Y = read_data(f"/Users/alexhagemeister/Downloads/csv/{dataset_name}.csv")
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

    # Run trials
    rmses = []
    training_times = []

    for _ in range(num_trials):
        rmse, training_time = train_and_evaluate(method, X_train, Y_train, X_test, Y_test)
        rmses.append(rmse)
        training_times.append(training_time)

    # Calculate the average RMSE and training time
    average_rmse = np.mean(rmses)
    average_training_time = np.mean(training_times)

    return average_rmse, average_training_time



In [ ]:
from highly_adaptive_lasso import HAL
from highly_adaptive_ridge import HAR
from kernel_har import KernelHAR
import numpy as np
import pandas as pd
from IPython.display import display
import time
import multiprocessing
import os

table_file_name = "models_compare_table.pickle"
table_df = pd.read_pickle(table_file_name)

for dataset_name in table_df['data']:
    method = HAR()
    rmse, training_time = run_real_data_trials(dataset_name, method)
    update_results(table_df, dataset_name, method.name, rmse, training_time)

    # Save the updated DataFrame
    table_df.to_pickle(table_file_name)

    # Display the updated DataFrame
    display(table_df)



,data,p,n,LTB,GBT,HAL,LASSO,HAR,KernelHAR
0,yacht,6,308,0.90 (10.69),0.90 (4.68),0.72 (0.92),8.92 (0.01),0.42 (0.49),–
1,energy,8,768,0.40 (30.82),0.40 (21.46),0.43 (45.80),4.14 (0.01),–,–
2,boston,13,506,3.35 (5.92),3.43 (4.47),3.66 (916.61),5.02 (0.01),–,–
3,concrete,8,1030,4.70 (43.29),4.87 (37.15),4.02 (134.01),10.40 (0.01),–,–
4,wine,11,1599,0.64 (6.01),0.63 (3.58),–,0.67 (0.03),–,–
5,power,4,9568,3.41 (56.92),3.46 (28.88),–,4.59 (0.04),–,–
6,kin8nm,8,8192,0.12 (96.50),0.10 (60.40),–,0.21 (0.03),–,–
7,naval,17,11934,0.00 (107.98),0.00 (56.19),–,0.01 (10.51),–,–
8,protein,9,45730,1.94 (611.38),1.94 (96.80),–,2.50 (0.33),–,–
9,blog,280,52397,23.49 (185.49),23.46 (9.90),–,28.25 (13.51),–,–


,data,p,n,LTB,GBT,HAL,LASSO,HAR,KernelHAR
0,yacht,6,308,0.90 (10.69),0.90 (4.68),0.72 (0.92),8.92 (0.01),0.42 (0.49),–
1,energy,8,768,0.40 (30.82),0.40 (21.46),0.43 (45.80),4.14 (0.01),0.43 (1.86),–
2,boston,13,506,3.35 (5.92),3.43 (4.47),3.66 (916.61),5.02 (0.01),–,–
3,concrete,8,1030,4.70 (43.29),4.87 (37.15),4.02 (134.01),10.40 (0.01),–,–
4,wine,11,1599,0.64 (6.01),0.63 (3.58),–,0.67 (0.03),–,–
5,power,4,9568,3.41 (56.92),3.46 (28.88),–,4.59 (0.04),–,–
6,kin8nm,8,8192,0.12 (96.50),0.10 (60.40),–,0.21 (0.03),–,–
7,naval,17,11934,0.00 (107.98),0.00 (56.19),–,0.01 (10.51),–,–
8,protein,9,45730,1.94 (611.38),1.94 (96.80),–,2.50 (0.33),–,–
9,blog,280,52397,23.49 (185.49),23.46 (9.90),–,28.25 (13.51),–,–


,data,p,n,LTB,GBT,HAL,LASSO,HAR,KernelHAR
0,yacht,6,308,0.90 (10.69),0.90 (4.68),0.72 (0.92),8.92 (0.01),0.42 (0.49),–
1,energy,8,768,0.40 (30.82),0.40 (21.46),0.43 (45.80),4.14 (0.01),0.43 (1.86),–
2,boston,13,506,3.35 (5.92),3.43 (4.47),3.66 (916.61),5.02 (0.01),3.79 (49.22),–
3,concrete,8,1030,4.70 (43.29),4.87 (37.15),4.02 (134.01),10.40 (0.01),–,–
4,wine,11,1599,0.64 (6.01),0.63 (3.58),–,0.67 (0.03),–,–
5,power,4,9568,3.41 (56.92),3.46 (28.88),–,4.59 (0.04),–,–
6,kin8nm,8,8192,0.12 (96.50),0.10 (60.40),–,0.21 (0.03),–,–
7,naval,17,11934,0.00 (107.98),0.00 (56.19),–,0.01 (10.51),–,–
8,protein,9,45730,1.94 (611.38),1.94 (96.80),–,2.50 (0.33),–,–
9,blog,280,52397,23.49 (185.49),23.46 (9.90),–,28.25 (13.51),–,–


,data,p,n,LTB,GBT,HAL,LASSO,HAR,KernelHAR
0,yacht,6,308,0.90 (10.69),0.90 (4.68),0.72 (0.92),8.92 (0.01),0.42 (0.49),–
1,energy,8,768,0.40 (30.82),0.40 (21.46),0.43 (45.80),4.14 (0.01),0.43 (1.86),–
2,boston,13,506,3.35 (5.92),3.43 (4.47),3.66 (916.61),5.02 (0.01),3.79 (49.22),–
3,concrete,8,1030,4.70 (43.29),4.87 (37.15),4.02 (134.01),10.40 (0.01),3.96 (3.69),–
4,wine,11,1599,0.64 (6.01),0.63 (3.58),–,0.67 (0.03),–,–
5,power,4,9568,3.41 (56.92),3.46 (28.88),–,4.59 (0.04),–,–
6,kin8nm,8,8192,0.12 (96.50),0.10 (60.40),–,0.21 (0.03),–,–
7,naval,17,11934,0.00 (107.98),0.00 (56.19),–,0.01 (10.51),–,–
8,protein,9,45730,1.94 (611.38),1.94 (96.80),–,2.50 (0.33),–,–
9,blog,280,52397,23.49 (185.49),23.46 (9.90),–,28.25 (13.51),–,–


,data,p,n,LTB,GBT,HAL,LASSO,HAR,KernelHAR
0,yacht,6,308,0.90 (10.69),0.90 (4.68),0.72 (0.92),8.92 (0.01),0.42 (0.49),–
1,energy,8,768,0.40 (30.82),0.40 (21.46),0.43 (45.80),4.14 (0.01),0.43 (1.86),–
2,boston,13,506,3.35 (5.92),3.43 (4.47),3.66 (916.61),5.02 (0.01),3.79 (49.22),–
3,concrete,8,1030,4.70 (43.29),4.87 (37.15),4.02 (134.01),10.40 (0.01),3.96 (3.69),–
4,wine,11,1599,0.64 (6.01),0.63 (3.58),–,0.67 (0.03),0.59 (107.46),–
5,power,4,9568,3.41 (56.92),3.46 (28.88),–,4.59 (0.04),–,–
6,kin8nm,8,8192,0.12 (96.50),0.10 (60.40),–,0.21 (0.03),–,–
7,naval,17,11934,0.00 (107.98),0.00 (56.19),–,0.01 (10.51),–,–
8,protein,9,45730,1.94 (611.38),1.94 (96.80),–,2.50 (0.33),–,–
9,blog,280,52397,23.49 (185.49),23.46 (9.90),–,28.25 (13.51),–,–


,data,p,n,LTB,GBT,HAL,LASSO,HAR,KernelHAR
0,yacht,6,308,0.90 (10.69),0.90 (4.68),0.72 (0.92),8.92 (0.01),0.42 (0.49),–
1,energy,8,768,0.40 (30.82),0.40 (21.46),0.43 (45.80),4.14 (0.01),0.43 (1.86),–
2,boston,13,506,3.35 (5.92),3.43 (4.47),3.66 (916.61),5.02 (0.01),3.79 (49.22),–
3,concrete,8,1030,4.70 (43.29),4.87 (37.15),4.02 (134.01),10.40 (0.01),3.96 (3.69),–
4,wine,11,1599,0.64 (6.01),0.63 (3.58),–,0.67 (0.03),0.59 (107.46),–
5,power,4,9568,3.41 (56.92),3.46 (28.88),–,4.59 (0.04),3.34 (83.11),–
6,kin8nm,8,8192,0.12 (96.50),0.10 (60.40),–,0.21 (0.03),–,–
7,naval,17,11934,0.00 (107.98),0.00 (56.19),–,0.01 (10.51),–,–
8,protein,9,45730,1.94 (611.38),1.94 (96.80),–,2.50 (0.33),–,–
9,blog,280,52397,23.49 (185.49),23.46 (9.90),–,28.25 (13.51),–,–


: 

In [ ]:
from kernel_har import KernelHAR
import numpy as np
import pandas as pd
from IPython.display import display
import time
import multiprocessing
import os

table_file_name = "models_compare_table.pickle"
table_df = pd.read_pickle(table_file_name)

for dataset_name in table_df['data']:
    method = KernelHAR()
    rmse, training_time = run_real_data_trials(dataset_name, method)
    update_results(table_df, dataset_name, method.name, rmse, training_time)

    # Save the updated DataFrame
    table_df.to_pickle(table_file_name)

    # Display the updated DataFrame
    display(table_df)

,data,p,n,LTB,GBT,HAL,LASSO,HAR,KernelHAR
0,yacht,6,308,0.90 (10.69),0.90 (4.68),0.72 (0.92),8.92 (0.01),0.42 (0.49),0.30 (9.80)
1,energy,8,768,0.40 (30.82),0.40 (21.46),0.43 (45.80),4.14 (0.01),0.43 (1.86),–
2,boston,13,506,3.35 (5.92),3.43 (4.47),3.66 (916.61),5.02 (0.01),3.79 (49.22),–
3,concrete,8,1030,4.70 (43.29),4.87 (37.15),4.02 (134.01),10.40 (0.01),3.96 (3.69),–
4,wine,11,1599,0.64 (6.01),0.63 (3.58),–,0.67 (0.03),0.59 (107.46),–
5,power,4,9568,3.41 (56.92),3.46 (28.88),–,4.59 (0.04),3.34 (83.11),–
6,kin8nm,8,8192,0.12 (96.50),0.10 (60.40),–,0.21 (0.03),–,–
7,naval,17,11934,0.00 (107.98),0.00 (56.19),–,0.01 (10.51),–,–
8,protein,9,45730,1.94 (611.38),1.94 (96.80),–,2.50 (0.33),–,–
9,blog,280,52397,23.49 (185.49),23.46 (9.90),–,28.25 (13.51),–,–


,data,p,n,LTB,GBT,HAL,LASSO,HAR,KernelHAR
0,yacht,6,308,0.90 (10.69),0.90 (4.68),0.72 (0.92),8.92 (0.01),0.42 (0.49),0.30 (9.80)
1,energy,8,768,0.40 (30.82),0.40 (21.46),0.43 (45.80),4.14 (0.01),0.43 (1.86),0.39 (30.21)
2,boston,13,506,3.35 (5.92),3.43 (4.47),3.66 (916.61),5.02 (0.01),3.79 (49.22),–
3,concrete,8,1030,4.70 (43.29),4.87 (37.15),4.02 (134.01),10.40 (0.01),3.96 (3.69),–
4,wine,11,1599,0.64 (6.01),0.63 (3.58),–,0.67 (0.03),0.59 (107.46),–
5,power,4,9568,3.41 (56.92),3.46 (28.88),–,4.59 (0.04),3.34 (83.11),–
6,kin8nm,8,8192,0.12 (96.50),0.10 (60.40),–,0.21 (0.03),–,–
7,naval,17,11934,0.00 (107.98),0.00 (56.19),–,0.01 (10.51),–,–
8,protein,9,45730,1.94 (611.38),1.94 (96.80),–,2.50 (0.33),–,–
9,blog,280,52397,23.49 (185.49),23.46 (9.90),–,28.25 (13.51),–,–


,data,p,n,LTB,GBT,HAL,LASSO,HAR,KernelHAR
0,yacht,6,308,0.90 (10.69),0.90 (4.68),0.72 (0.92),8.92 (0.01),0.42 (0.49),0.30 (9.80)
1,energy,8,768,0.40 (30.82),0.40 (21.46),0.43 (45.80),4.14 (0.01),0.43 (1.86),0.39 (30.21)
2,boston,13,506,3.35 (5.92),3.43 (4.47),3.66 (916.61),5.02 (0.01),3.79 (49.22),3.39 (15.92)
3,concrete,8,1030,4.70 (43.29),4.87 (37.15),4.02 (134.01),10.40 (0.01),3.96 (3.69),–
4,wine,11,1599,0.64 (6.01),0.63 (3.58),–,0.67 (0.03),0.59 (107.46),–
5,power,4,9568,3.41 (56.92),3.46 (28.88),–,4.59 (0.04),3.34 (83.11),–
6,kin8nm,8,8192,0.12 (96.50),0.10 (60.40),–,0.21 (0.03),–,–
7,naval,17,11934,0.00 (107.98),0.00 (56.19),–,0.01 (10.51),–,–
8,protein,9,45730,1.94 (611.38),1.94 (96.80),–,2.50 (0.33),–,–
9,blog,280,52397,23.49 (185.49),23.46 (9.90),–,28.25 (13.51),–,–


,data,p,n,LTB,GBT,HAL,LASSO,HAR,KernelHAR
0,yacht,6,308,0.90 (10.69),0.90 (4.68),0.72 (0.92),8.92 (0.01),0.42 (0.49),0.30 (9.80)
1,energy,8,768,0.40 (30.82),0.40 (21.46),0.43 (45.80),4.14 (0.01),0.43 (1.86),0.39 (30.21)
2,boston,13,506,3.35 (5.92),3.43 (4.47),3.66 (916.61),5.02 (0.01),3.79 (49.22),3.39 (15.92)
3,concrete,8,1030,4.70 (43.29),4.87 (37.15),4.02 (134.01),10.40 (0.01),3.96 (3.69),3.88 (70.02)
4,wine,11,1599,0.64 (6.01),0.63 (3.58),–,0.67 (0.03),0.59 (107.46),–
5,power,4,9568,3.41 (56.92),3.46 (28.88),–,4.59 (0.04),3.34 (83.11),–
6,kin8nm,8,8192,0.12 (96.50),0.10 (60.40),–,0.21 (0.03),–,–
7,naval,17,11934,0.00 (107.98),0.00 (56.19),–,0.01 (10.51),–,–
8,protein,9,45730,1.94 (611.38),1.94 (96.80),–,2.50 (0.33),–,–
9,blog,280,52397,23.49 (185.49),23.46 (9.90),–,28.25 (13.51),–,–


,data,p,n,LTB,GBT,HAL,LASSO,HAR,KernelHAR
0,yacht,6,308,0.90 (10.69),0.90 (4.68),0.72 (0.92),8.92 (0.01),0.42 (0.49),0.30 (9.80)
1,energy,8,768,0.40 (30.82),0.40 (21.46),0.43 (45.80),4.14 (0.01),0.43 (1.86),0.39 (30.21)
2,boston,13,506,3.35 (5.92),3.43 (4.47),3.66 (916.61),5.02 (0.01),3.79 (49.22),3.39 (15.92)
3,concrete,8,1030,4.70 (43.29),4.87 (37.15),4.02 (134.01),10.40 (0.01),3.96 (3.69),3.88 (70.02)
4,wine,11,1599,0.64 (6.01),0.63 (3.58),–,0.67 (0.03),0.59 (107.46),0.60 (520.56)
5,power,4,9568,3.41 (56.92),3.46 (28.88),–,4.59 (0.04),3.34 (83.11),–
6,kin8nm,8,8192,0.12 (96.50),0.10 (60.40),–,0.21 (0.03),–,–
7,naval,17,11934,0.00 (107.98),0.00 (56.19),–,0.01 (10.51),–,–
8,protein,9,45730,1.94 (611.38),1.94 (96.80),–,2.50 (0.33),–,–
9,blog,280,52397,23.49 (185.49),23.46 (9.90),–,28.25 (13.51),–,–


: 